In [14]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from dotenv import load_dotenv
import os

In [15]:
load_dotenv()

True

In [16]:
##### GET THE DATE AND COMPARE WITH TODAT

# URL of the webpage you want to scrape
url = 'https://www.cgeonline.com.ar/informacion/apertura-de-citas.html'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> elements
td_elements = soup.find_all('td')

# Search for the <td> element containing 'Visados'
for index, td in enumerate(td_elements):
    if td.get_text(strip=True) == 'Visados':
        # Get the next <td> element
        date_td = td_elements[index + 2]  # Index + 2 for the second <td> after 'Visados'
        date_text = date_td.get_text(strip=True)
        break

# Extract the date part from the string
date_part = date_text.split()[1]  # '04/04/2024'

# Parse the date string to a datetime object
date_object = datetime.strptime(date_part, '%d/%m/%Y').date()

# Get today's date
today = datetime.now().date()

In [17]:
# Function to send email
def send_email(subject, body):
    # Email configuration
    sender_email = os.getenv('python_sender')
    receiver_email = os.getenv('python_receiver')
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_username = os.getenv('python_sender')
    smtp_password = os.getenv('python_pass')
    
    # Create a MIME object
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the body of the email
    msg.attach(MIMEText(body, 'plain'))
    
    # Connect to the SMTP server and send the email
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

In [18]:
# Compare dates
if date_object < today:
    # Date is before today
    subject = 'No news! 👎'
    body = f'The last day so far was: {date_text}'
else:
    # Date is today or after
    subject = 'New dates !!!!! 🎉'
    body = f'The new date is: {date_text}'

# Send email
send_email(subject, body)